# Production Planning Problem
This is a very simple production planning problem.

Suppose that we are responsible for scheduling the monthly production plan of a product for a year. Here are some assumptions:
- The demand of the product, unit production cost, and production capacity in each month are given.
- Inventory holding costs are assessed at the end of each month.
- Holding cost per unit is invariant from month to month.
- There are 500 units of inventory available at the beginning of the first month.
- No shortage is allowed.

### Mathematical Formulation
Let us derive the formulation from the problem description.

##### Sets:
- **t**: the set of time periods (months)

##### Parameters:
- **h**: unit holding cost
- **p**: production capacity per month
- **I_0**: the initial inventory
- **c_t**ₜ: unit production cost in month t
- **d_t**: demand of month t

##### Variables:
- **X_t**: amount produced in month t
- **I_t**: inventory at the end of period t

##### Objective:
**Minimize: (h x I_t + c_t x X_t)**

###### Constraints
- Inventory Constraints: I_{t-1} + X_t - d_t = I_t
- Capacity Constraints: X_t <= p

In [1]:
# Import PuLP modeler functions
from pulp import *
import pandas as pd
from time import time

In [2]:
# Create a dictionary from the available data
input_df_dict = {}
input_df_dict['input_data'] = pd.read_csv("input_data.csv")
input_df_dict['parameters'] = pd.read_csv("parameters.csv")
input_df_dict

{'input_data':     period  demand  production_cost  production_capacity
 0        1    3300              150                 6400
 1        2    3300              150                 6400
 2        3    4500              140                 6800
 3        4    6100              140                 6800
 4        5    5100              150                 6400
 5        6    9000              135                 7000
 6        7    9700              135                 7000
 7        8    8200              135                 7000
 8        9    7900              135                 6800
 9       10    5700              140                 6800
 10      11    4500              140                 6800
 11      12    4200              140                 6400,
 'parameters':            attribute  value
 0       holding_cost      8
 1  initial_inventory    500}

In [3]:
# As the parameters are few, a dictionary is created separately
input_param_dict = input_df_dict['parameters'].set_index('attribute')['value'].to_dict()
input_param_dict

{'holding_cost': 8, 'initial_inventory': 500}

In [4]:
# Create the model variable to contain the problem data
model = LpProblem(name='Production_Planning', sense=LpMinimize)

In [5]:
# Decision variables
production_variables = LpVariable.dicts(name='X', indexs=input_df_dict['input_data'].index,
                                        lowBound=0, cat=LpContinuous)
# i.e, {0: X_0, 1: X_1, 2: X_2, 3: X_3, 4: X_4, 5: X_5, 6: X_6, 7: X_7, 8: X_8, 9: X_9, 10: X_10, 11: X_11}

inventory_variables = LpVariable.dicts(name='I', indexs=input_df_dict['input_data'].index,
                                       lowBound=0, cat=LpContinuous)
# i.e, {0: I_0, 1: I_1, 2: I_2, 3: I_3, 4: I_4, 5: I_5, 6: I_6, 7: I_7, 8: I_8, 9: I_9, 10: I_10, 11: I_11}

In [6]:
production_variables, inventory_variables

({0: X_0,
  1: X_1,
  2: X_2,
  3: X_3,
  4: X_4,
  5: X_5,
  6: X_6,
  7: X_7,
  8: X_8,
  9: X_9,
  10: X_10,
  11: X_11},
 {0: I_0,
  1: I_1,
  2: I_2,
  3: I_3,
  4: I_4,
  5: I_5,
  6: I_6,
  7: I_7,
  8: I_8,
  9: I_9,
  10: I_10,
  11: I_11})

In [7]:
# Inventory balance constraints
for period, value in input_df_dict['input_data'].iloc[1:].iterrows():
    model.addConstraint(LpConstraint(
        e=inventory_variables[period - 1] + production_variables[period] - inventory_variables[period],
        sense=LpConstraintEQ,
        name='inv_balance' + str(period),
        rhs=value.demand))
#i.e,    1 I_0 - I_1 + X_1 = 3300
#        2 I_1 - I_2 + X_2 = 4500
#        3 I_2 - I_3 + X_3 = 6100
#        4 I_3 - I_4 + X_4 = 5100
#        5 I_4 - I_5 + X_5 = 9000
#        6 I_5 - I_6 + X_6 = 9700
#        7 I_6 - I_7 + X_7 = 8200
#        8 I_7 - I_8 + X_8 = 7900
#        9 I_8 - I_9 + X_9 = 5700
#        10 -I_10 + I_9 + X_10 = 4500
#        11 I_10 - I_11 + X_11 = 4200

# inv balance for first period
model.addConstraint(LpConstraint(
    e=production_variables[0] - inventory_variables[0],
    sense=LpConstraintEQ,
    name='inv_balance0',
    rhs=input_df_dict['input_data'].iloc[0].demand - input_param_dict['initial_inventory']))
# i.e, (-1*I_0 + 1*X_0 + 0 = 3300 - 500)

In [8]:
# Production capacity constraints
for index, value in production_variables.items():
    model.addConstraint(LpConstraint(
        e=value,
        sense=LpConstraintLE,
        name='prod_cap_month_' + str(index),
        rhs=input_df_dict['input_data'].iloc[index].production_capacity))
# i.e,   X_0 = 6400
#        X_1 = 6400
#        X_3 = 6800
#        X_4 = 6400
#        X_5 = 7000
#        X_6 = 7000
#        X_7 = 7000
#        X_8 = 6800
#        X_9 = 6800
#        X_10 = 6800
#        X_11 = 6400

In [9]:
# Costs and objective function
total_holding_cost = input_param_dict['holding_cost'] * lpSum(inventory_variables)
# i.e, 8*I_0 + 8*I_1 + 8*I_10 + 8*I_11 + 8*I_2 + 8*I_3 + 8*I_4 + 8*I_5 + 8*I_6 + 8*I_7 + 8*I_8 + 8*I_9 + 0

total_production_cost = lpSum(row['production_cost'] * production_variables[index]
                              for index, row in input_df_dict['input_data'].iterrows())
# i.e, 150*X_0 + 150*X_1 + 140*X_10 + 140*X_11 + 140*X_2 + 140*X_3 + 150*X_4 + 135*X_5 + 135*X_6 + 135*X_7 + 135*X_8 + 140*X_9 + 0

objective = total_holding_cost + total_production_cost

model.setObjective(objective)

Model creation time in sec: 0.4643


In [10]:
model.objective

8*I_0 + 8*I_1 + 8*I_10 + 8*I_11 + 8*I_2 + 8*I_3 + 8*I_4 + 8*I_5 + 8*I_6 + 8*I_7 + 8*I_8 + 8*I_9 + 150*X_0 + 150*X_1 + 140*X_10 + 140*X_11 + 140*X_2 + 140*X_3 + 150*X_4 + 135*X_5 + 135*X_6 + 135*X_7 + 135*X_8 + 140*X_9 + 0

In [11]:
model.name

'Production_Planning'

In [12]:
model.solve()
print(model.status, LpStatus[model.status])

1 Optimal


In [13]:
# Each of the variables is printed with it's resolved optimum value
for v in model.variables():
    print(v.name, "=", v.varValue)

I_0 = 0.0
I_1 = 2700.0
I_10 = 0.0
I_11 = 0.0
I_2 = 5000.0
I_3 = 5700.0
I_4 = 7000.0
I_5 = 5000.0
I_6 = 2300.0
I_7 = 1100.0
I_8 = 0.0
I_9 = 0.0
X_0 = 2800.0
X_1 = 6000.0
X_10 = 4500.0
X_11 = 4200.0
X_2 = 6800.0
X_3 = 6800.0
X_4 = 6400.0
X_5 = 7000.0
X_6 = 7000.0
X_7 = 7000.0
X_8 = 6800.0
X_9 = 5700.0


In [14]:
# The optimised objective function value is printed to the screen    
print("Total Cost of Production = ", model.objective.value())

Total Cost of Production =  10183400.0
